In [ ]:
import bs4
import time
from datetime import datetime
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By


In [ ]:
def get_article_html(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    time.sleep(5) 
    return soup

In [ ]:
def get_data_abc_web(soup):   
    all_articles_part1= soup.find_all('div',attrs={'class':'yqrQw'})
    all_articles_part2 = soup.find_all('div',attrs={'class':'_2I1aj'})
    
    # Declaration
    articles_topic = []
    articles_content = []
   
    for article in all_articles_part1:
        topic = article.find('a').get_text()
        articles_topic.append(topic)

        url_of_article = article.find('a')['href']
        soup = get_article_html("https://www.abc.net.au/" + url_of_article)

        content = soup.find_all("p")
        full_content = ""
        if content != None:
            for paragraph in content:
                full_content = full_content + " " + paragraph.get_text().strip()
        articles_content.append(full_content) 
        
    for article in all_articles_part2:
        if article.find('a') != None:
            topic = article.find('a').get_text()
        
            if article.find('a')['href'] != None:
                url_of_article = article.find('a')['href']
                soup = get_article_html("https://www.abc.net.au/" + url_of_article)

                content = soup.find_all("p")
                full_content = ""
                if content != None:
                    for paragraph in content:
                        full_content = full_content + " " + paragraph.get_text().strip()
                articles_topic.append(topic)
                articles_content.append(full_content) 
      
    return articles_topic, articles_content

In [ ]:
def insert_data_to_csv(topic, category, content):
    # Create dataframe
    data = pd.DataFrame({
    "topic": topic,
    "category": category,
    "content": content
    })
    data.to_csv(f'abc Scraping-{category}-{datetime.now().strftime("%d-%m-%Y")}.csv', mode='a')

In [ ]:
categories = ["politics", "analysis-and-opinion", "arts-culture", "environment", "house-and-home",
              "travel-and-tourism-lifestyle-and-leisure", "shopping-mall", "markets", "society"
              "business", "world", "health", "sport", "science",]

print("Start scraping from abc!")

def scraping_with_selenium_from_url(url):
        driver = webdriver.Chrome()        
        driver.get(url)
        driver.maximize_window()
        time.sleep(5)
        close_cookies= "//button[@class='_1KwgR _1uCkA _2vzFN _20SSK eCXal _2VqCY pigdr']"
        driver.find_element(by=By.XPATH, value=close_cookies).click()
        time.sleep(5)
        for i in range (1, 20):
            next_xpath = "//button[@class='_1KwgR _2-5J1 eCXal']"
            driver.find_element(by=By.XPATH, value=next_xpath).click() 
            time.sleep(3)
        html = driver.page_source
        driver.close()
        soup = BeautifulSoup(html,"html.parser")
        topic, content = get_data_abc_web(soup)
        insert_data_to_csv(topic, category, content)
    
for category in categories:
    if category in ("environment", "house-and-home", "travel-and-tourism-lifestyle-and-leisure", "shopping-mall",  "markets", "society"):
        scraping_with_selenium_from_url(f"https://www.abc.net.au/news/topic/{category}")
    else:
        scraping_with_selenium_from_url(f"https://www.abc.net.au/news/{category}")
        
print("Finish scraping from abc.net.au!")
